# MV Lidar Tutorial

This notebook aims to provide a simple tutorial for training using the MV Lidar model.

In [24]:
#| eval: false
from semantic_kitti_implementations import mvlidar_model
from semantic_kitti_implementations import train
from semantic_kitti_implementations import data

from torch.nn import CrossEntropyLoss
from torchvision import transforms
from torch.optim import Adam

import wandb

For training management purposes, wandb is being used, [click here](https://docs.wandb.ai/quickstart) for a quickstart if you need one. \
Adjust the following hyperparameters below in whatever way you find most suitable.

In [34]:
#| eval: false
wandb.init(
    project="mv_lidar", # name of the project
    name="v4", # name of the experiment
    dir="../", # directory for wandb artifacts

    config={
        "learning_rate": 0.01,
        "num_epochs": 100,
        "batch_size": 4,
        "num_classes": 7
    }
)

Define the paths for the images and annotations. It's worth to know that the model takes as input a two-channel image containing reflectance and depth. For more information, visit the [Cloud2dImageConverter](https://github.com/AIR-UFG/Cloud2DImageConverter) repository.

In [35]:
train_imgs_path = "../dataset/train/imgs/"
test_imgs_path =  "../dataset/test/imgs/"
train_anns_path = "../dataset/train/anns/"
test_anns_path = "../dataset/test/anns/"

Since this model does not utilize all of the Semantic Kitti classes, we will be remapping the ones that we will use and removing the others.

In [36]:
#| eval: false
remapping_rules = {
  1: 1,
  4: 2,
  6: 3,
  7: 4,
  2: 4,
  9: 5,
  11: 6
}

data.remap_segmentation_masks(train_anns_path, train_anns_path, remapping_rules=remapping_rules)
data.remap_segmentation_masks(test_anns_path, test_anns_path, remapping_rules=remapping_rules)

In [37]:
#| eval: false
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = data.SemanticDataset(image_path=train_imgs_path,mask_path=train_anns_path,transform=transform)
test_dataset = data.SemanticDataset(image_path=test_imgs_path,mask_path=test_anns_path,transform=transform)

In [38]:
evaluator = train.Evaluator(loss_fn=CrossEntropyLoss(reduction="none"))

In [39]:
learner = train.Learner(model=mvlidar_model.MVLidar, 
                        optimizer=Adam, 
                        lr=wandb.config["learning_rate"], 
                        num_classes=wandb.config["num_classes"])

In [40]:
trainer = train.Trainer(train_dataset=train_dataset, 
                        test_dataset=test_dataset, 
                        learner=learner, 
                        evaluator=evaluator, 
                        batch_size=wandb.config["batch_size"])

In [41]:
trainer.run(wandb=wandb,
            n_epochs=wandb.config["num_epochs"])

Epoch 20
------------
Train loss 0.00018572086992207915
Test loss 0.0050450339913368225
Epoch 40
------------
Train loss 6.047436727385502e-05
Test loss 0.00014661295426776633
Epoch 60
------------
Train loss 3.628457125159912e-05
Test loss 6.894755642861128e-05
Epoch 80
------------
Train loss 2.4496363948856015e-05
Test loss 3.9093098166631535e-05
Epoch 100
------------
Train loss 0.00018958958389703184
Test loss 0.002348897571209818
Done!


In [42]:
wandb.finish()

test_loss,▃▂▇▆██▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
train_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
test_loss,0.00235
train_loss,0.00019
